## Log

Bigram: 2.4716, 2.4755




In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
txt = open('shakespeare.txt', 'r').read()
len(txt)

5447119

In [3]:
txt[0:500]

"                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud"

In [4]:
chars = list(set(txt))
chars.sort()

ctoi = {c:i for i, c in enumerate(chars)}
itoc = {i:c for i, c in enumerate(chars)}
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)


 !"&'(),-.0123456789:;<>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz|}
84


In [5]:
i =  math.floor(0.9 * len(txt))
train_txt = txt[0:i]
valid_txt = txt[i+1:]

len(train_txt), len(valid_txt)

(4902407, 544711)

In [6]:
train_tkns = [ctoi[c] for c in train_txt]
valid_tkns = [ctoi[c] for c in valid_txt]

In [7]:
block_size = 8
batch_size = 32

def txt_to_token(t):
    return [ctoi[c] for c in t]
    
# (B, L)
def random_batch():
    xi = torch.randint(0, len(train_tkns)-block_size, (batch_size,))
    x = torch.tensor([train_tkns[i:i+block_size] for i in xi])
    y = torch.tensor([train_tkns[i+1:i+block_size+1] for i in xi])
    
    return x, y

x, y = random_batch()
x.shape

torch.Size([32, 8])

In [8]:
x[0]

tensor([74,  8,  0,  1,  1,  1,  1, 29])

In [9]:
class BigramModel(nn.Module):    
    
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, vocab_size)

    # (B, L) -> (B, L, C)
    def forward(self, x):
        y = self.embed(x)
        return y
            
bi_model = BigramModel()
bi_model(x).shape

torch.Size([32, 8, 84])

In [10]:
def eval_split(split, model):
    tkn = train_tkns if split == "train" else valid_tkns
    tkn = torch.tensor(tkn)

    bsize = len(tkn) // block_size
    x = tkn[0:bsize*block_size]
    y = tkn[1:bsize*block_size+1]
    x = x.view(bsize, block_size) # (B, L)
    y = y.view(bsize, block_size) # (B, L)
    
    logits = model(x) # (B, L, C)
    B, L, C = logits.shape
    loss = F.cross_entropy(logits.view(B*L, C), y.view(B*L))
    
    return loss.item()

In [11]:
def sample(model):
    max_len = 500
    tks = [0]*block_size

    for i in range(max_len):
        ctx = torch.tensor(tks[i:i+block_size]) # (L)
        ctx = ctx.view(1, -1) # (B, L)

        logits = model(ctx) # (B, L, C)
        probs = F.softmax(logits, dim=2) # (B, L, C)
        probs = probs[0,-1,:] # (C), # the last in the sequence is the newly generated
        yi = torch.multinomial(probs, 1)
        tks.append(yi.item())

    chars = [itoc[t] for t in tks]
    return "".join(chars)

In [12]:
print(sample(bi_model))









P'Gs)G 7-Ucr!y3}"4M8OV`ngHJ)B7'GGJ:P!Yu[zo9zi<_T|pMivr;uhhe8w"I,)Yb-PE2Dq>QH,rC
i
lWwbMTY9I?rMTEiHzY|c:gn
!VXNuXdah)e?-YuMh`5;2ah;6|!CF|Hu
o8"7g :(nnX8U4V,d k'G>Ta?!MVffMD&[u"!uV3iA?&(r;|y6tp"3bPq"L1TX BYS51Tnad8Lxy4>W[ZoKM m'`J1:7>5x]ZEjGu}-uOk-NttJ0 ZuC>|_upq;|'[-nl!_VIj'F}EXVM!ACGLYH(oV4C,r;6he]"tlg3U<2li.twj&")wqAQzbqMTM5j&'}xXX]P >ORcS>!6K, t)T3Gu0R754C,y6o&V>.x)krcb'R)j'94q"Gdbr>?1a5:S6Sm'Gw:&|"3v ietz1Lfr-9VS?w<2D5TVI;Ph-CS7WTHZnDP4,<aR?Ql3S06
fZy6<6f;okU61nVub1zXv:B}G4CAvBK5BLYu>W8'MXO]3


In [13]:
def fit(model):
    optim = torch.optim.Adam(model.parameters())
    
    for i in range(50000):
        optim.zero_grad()

        xb, yb = random_batch()
        logits = model(xb) # (B, L, C)

        B, L, C = logits.shape
        loss = F.cross_entropy(logits.view(B*L, C), yb.view(B*L))

        loss.backward()
        optim.step()

        if i % 5000 == 0:
            print(f"{loss.item():.4f}")

In [14]:
fit(bi_model)

4.8498
2.5652
2.4412
2.4191
2.4259
2.3939
2.5491
2.4699
2.4002
2.4414


In [16]:
tr_loss = eval_split("train", bi_model)
va_loss = eval_split("valid", bi_model)

print(f"train: {tr_loss:.4f}")
print(f"valid: {va_loss:.4f}")
print(f"baseline: {-torch.tensor(1/vocab_size).log():.4f}")

train: 2.4709
valid: 2.4882
baseline: 4.4308


In [17]:
print(sample(bi_model))









Whe.
  D s  I  bthartrr PUntexpuseelo, hoke   Fofamok 'sspl r Sour fre'st br CRECALOSAR sear   istured I nera TER NINRe.
 by at  l Inoumo  bo'd.
 nok  th' cakin.
 'd, ans han whove Carthan  spoint,  MANDERACOLinop's,
 to;
 m chalor   o k han.  whe  THICoolat ct Ist y
 whorithurdond
Anteds fo  mer EOENTHens  seyo Qus
 BUKice beth h'd  ounge  wagouthak lit ore,   itaps pre    ththt  dsinds!

 he BAn'd ARithonof!
  sheleain h!

  
  te, wouk m.  m  aitome'lait F   watoung d-in an  meng nt TCENDWiru


In [18]:
# Attension mechanism

head_size = 5
x = torch.rand(4, block_size, vocab_size) # (B, L, C)

key = nn.Linear(vocab_size, head_size)
query = nn.Linear(vocab_size, head_size)
value = nn.Linear(vocab_size, head_size)

k = key(x)   # (B, L, C)
q = query(x) # (B, L, C)
v = value(x) # (B, L, C)

q = q.permute(0, 2, 1) # (B, C, L)
w = k @ q # (B, L, L)

B, L, C = k.shape
mask = torch.tril(torch.ones(L, L))
mask = mask == 0
w = w.masked_fill(mask, -float('inf'))

prob = F.softmax(w, dim=2) # (B, L, L) TODO: not sure which dimension
a = prob @ v # (B, L, L) @ # (B, L, C)
a.shape # (B, L, C)

torch.Size([4, 8, 5])

In [19]:
class Attention(nn.Module):    
    
    def __init__(self, vocab_size, head_size):
        super().__init__()
        
        self.head_size = head_size
        self.key = nn.Linear(vocab_size, head_size)
        self.query = nn.Linear(vocab_size, head_size)
        self.value = nn.Linear(vocab_size, head_size)
        
    # (B, L, C)  ->  (B, L, C')  
    def forward(self, x):
        k = self.key(x)   # (B, L, C')
        q = self.query(x) # (B, L, C')
        v = self.value(x) # (B, L, C')
        
        # TODO: other way than permute?
        q = q.permute(0, 2, 1) # (B, C', L)
        w = k @ q # (B, L, L)
        w /= self.head_size**0.5
        
        B, L, C = k.shape
        mask = torch.tril(torch.ones(L, L))
        mask = mask == 0
        w = w.masked_fill(mask, -float('inf'))

        prob = F.softmax(w, dim=2) # (B, L, L)
        a = prob @ v # (B, L, C')

        return a

In [20]:
hidden_size = 100
head_size = 50

class Transformer(nn.Module):    
    
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.attn = Attention(hidden_size, head_size)
        self.linear = nn.Linear(head_size, vocab_size)

    # (B, L) -> (B, L, C)
    def forward(self, x):
        y = self.embed(x) # (B, L, C)
        y = self.attn(y) # (B, L, C)
        y = self.linear(y) # (B, L, C)
        
        return y

In [21]:
tf_model = Transformer()

In [22]:
fit(tf_model)

4.4656
2.2867
2.2481
2.3425
2.3237
2.3785
2.4281
2.3291
2.6013
2.4782


In [23]:
tr_loss = eval_split("train", tf_model)
va_loss = eval_split("valid", tf_model)

print(f"train: {tr_loss:.4f}")
print(f"valid: {va_loss:.4f}")

train: 2.3899
valid: 2.4041


In [ ]:
print(sample(tf_model))